# Direcionamentos

- Primeiro, dar uma geral nas bibliotecas e documentação
- Dar uma passada na Cartilha de GTFS que está na tabela de atividades e, eventualmente, na [documentação oficial](https://gtfs.org/documentation/schedule/reference/)
- Implementar sequência lógica das análises
    - Separar bem o notebook como mostrado em títulos e subtitulos etc
    - Acrescentar textos explicativos breves
    - Pode ajudar ver [as postagens no Medium de Santiago Toso](https://medium.com/swlh/python-for-transit-get-the-most-out-of-your-gtfs-with-this-python-package-44d0b732f657), mas desde a postagem houve leves alterações no código-fonte
    - Usar [GTFS do Rio de Janeiro](https://www.data.rio/datasets/gtfs-do-rio-de-janeiro/about)
        - O GTFS pode estar "frequency-based", o que pode exigir que seja feito um [processamento preliminar](https://gist.github.com/invisiblefunnel/6c9f3a9b537d3f0ad192c24777b6ae57)

# Leitura e Análises Preliminares

Inclui leitura de arquivos GTFS utilizando [Partridge](https://github.com/remix/partridge) e [GTFS Functions](https://github.com/Bondify/gtfs_functions)

Autor: [Rede Mob](https://www.redemob.com.br/)

In [1]:
# Desativa alguns avisos e a impressão de log de
# eventos (registros de atividades) das bibliotecas
# utilizadas. GTFS functions, em particular pode ser
# bem "verborrágica". Experimente não rodar essa
# célula para ver o que acontece. O código ainda
# funcionará, mas com a impressão de algumas mensagens
# adicionais eventualmente.

import warnings
warnings.filterwarnings("ignore")

import logging, sys
logging.disable(sys.maxsize)

# Frequency to equivalent trips

In [2]:
from copy import copy
import math

from networkx.utils import pairwise
import numpy as np
import pandas as pd
import partridge as ptg


def seconds_to_gtfs_time(total_seconds):
    if math.isnan(total_seconds):
        return total_seconds  # TODO: What to do here?
    minutes, seconds = divmod(total_seconds, 60)
    hours, minutes = divmod(minutes, 60)
    time = list(map(lambda x: str(x).rjust(2, '0'), [int(hours), int(minutes), int(seconds)]))
    return f'{time[0]}:{time[1]}:{time[2]}'

inpath = "gtfs_rio_2014_moovit.zip"

feed = ptg.load_feed(inpath)

trips_by_id = {}
for _, trip in feed.trips.iterrows():
    trips_by_id[trip.trip_id] = dict(trip)

trip_patterns = {}
for trip_id, stop_times in feed.stop_times.sort_values("stop_sequence").groupby("trip_id"):
    stops = tuple(stop_times.stop_id)
    mintime = stop_times.arrival_time.min()
    times = tuple(t - mintime for t in stop_times.arrival_time)
    trip_patterns[trip_id] = (stops, times)

freq_trips = []
for _, freq in feed.frequencies.iterrows():
    window_start = int(freq.start_time)
    window_end = int(freq.end_time)
    for start in range(window_start, window_end, freq.headway_secs):
        freq_trips.append({
            "trip_id": freq.trip_id,
            "start": start,
        })

new_trips = []
new_stop_times = []
for i, ftrip in enumerate(freq_trips, start=1):
    new_trips.append(copy(trips_by_id[ftrip["trip_id"]]))
    new_trips[-1]["trip_id"] = i # override trip_id

    stops, times = trip_patterns[ftrip["trip_id"]]
    for j in range(len(stops)):
        t = seconds_to_gtfs_time(times[j] + ftrip["start"])
        new_stop_times.append({
            "trip_id": i,
            "stop_id": stops[j],
            "arrival_time": t,
            "departure_time": t,
            "stop_sequence": j + 1,
        })

trips_df = pd.DataFrame(new_trips)
stop_times_df = pd.DataFrame(new_stop_times)
empty_frequencies_df = ptg.utilities.empty_df()

new_feed = ptg.load_raw_feed(inpath)
new_feed.set("trips.txt", trips_df)
new_feed.set("stop_times.txt", stop_times_df)
new_feed.set("frequencies.txt", empty_frequencies_df) # we don't want frequencies.txt

ptg.writers.write_feed_dangerously(new_feed, "gtfs_rio_trip_based.zip")



ValueError: File or path not found: gtfs_rio_2014_moovit.zip

In [ ]:
import partridge as ptg
feed = ptg.load_geo_feed('gtfs_rio_2014_moovit.zip')

In [ ]:
feed.stop_times

# GTFS Functions

Cosulte o [repositório](https://github.com/Bondify/gtfs_functions) da biblioteca para maior detalhamento dos procedimentos aqui utilizados. Importante salientar que, lá, para algumas das visualizações de mapas é utilizada a biblioteca [Kepler](https://kepler.gl/). Aqui, utilizamos o próprio [geopandas](https://geopandas.org/en/stable/) para produzir os mapas porque a Kepler pode não funcionar bem no jupyter lab, aqui utilizado neste exemplo.

In [3]:
from gtfs_functions import Feed
from gtfs_functions.gtfs_plots import map_gdf

# Biblioteca de visualizações
import plotly.graph_objects as go
import plotly.express as px

ImportError: cannot import name 'geo_to_h3' from 'h3' (C:\Users\brand\anaconda3\envs\roda\Lib\site-packages\h3\__init__.py)

In [ ]:
# Leitura do arquivo GTFS
feed = Feed(
    gtfs_path='gtfs_rio_trip_based.zip',
    time_windows=[0, 6, 9, 15, 19, 22, 24],
    busiest_date=True,
    )

In [ ]:
# Calcula o tempo entre veículos
# para cada ponto de parada
stop_freq = feed.stops_freq

stop_freq.head()

In [ ]:
# Sentido das viagens: sentido centro, p. ex.
condition_dir = stop_freq.direction_id == 0

# Janela horária da análise
condition_window = stop_freq.window == '6:00-9:00'

# Filtrando os dados de acordo com sentido e janela horária
gdf = stop_freq.loc[(condition_dir & condition_window),:].reset_index()

# Função de mapeamento da biblioteca GTFS Functions
map_gdf(
    # Base com viagens filtradas
    gdf=gdf, 
    # Coluna com número de viagens por ponto
    variable='ntrips', 
    # Agrupando a quantidade de viagens em intervalos
    # para visualuzação no mapa temático
    breaks=[10, 20, 30, 40, 120, 200],
    # Cores de cada intervalo
    colors=["#d13870", "#e895b3" ,'#55d992', '#3ab071', '#0e8955','#066a40'], 
    tooltip_var=['min_per_trip'] , 
    tooltip_labels=['Frequency: '],   
    )

In [ ]:
# Cálculo das frequências por linha e trecho
segments_freq = feed.segments_freq

segments_freq.head(3)

In [ ]:
# Filtragem
mask = (
    (segments_freq.route_id == 'ALL_LINES')
    & (segments_freq.window == '15:00-19:00')
    )

gdf = segments_freq.loc[mask, :].reset_index()

# Construção do mapa
(
    gdf
    .sort_values('min_per_trip')
    .explore(
        column='min_per_trip',
        cmap='coolwarm',
        scheme='UserDefined',
        tiles='Carto DB positron',
        classification_kwds={
            'bins': [15, 30, 45, 60] ,       
            }
        )
    )

In [ ]:
# Cálculo das velocidades médias por trecho
speeds = feed.avg_speeds

speeds.head()

In [ ]:
# Filtragem
mask = (
    (speeds.route_id == 'ALL_LINES')
    & (speeds.window == '6:00-9:00')
    )

gdf = speeds.loc[mask, :].reset_index()

# Construção do mapa
(
    gdf
    .sort_values('speed_kmh')
    .explore(
        column='speed_kmh',
        cmap='coolwarm_r',
        scheme='NaturalBreaks',
        tiles='Carto DB positron'
        )
    )

In [ ]:
# Mapa de calor das velocidades ao longo do itinerário:
# Visualização das velocidades entre cada um dos pontos
# parada da linha

# Filtragem
mask = (speeds.direction_id == 1) & (speeds.route_name == '76 Hall/Greenburg')
dir_0 = speeds.loc[mask].sort_values(by='stop_sequence') 

# Perfil de velocidades ao longo de todas
# as janelas horárias
dir_0['hour'] = dir_0.window.apply(lambda x: int(x.split(':')[0]))
dir_0 = dir_0.sort_values(by=['hour', 'stop_sequence'], ascending=True)

fig = go.Figure(
    data=go.Heatmap(
        z=dir_0.speed_kmh, y=dir_0.start_stop_name, x=dir_0.window,
        hoverongaps = False, colorscale=px.colors.colorbrewer.RdYlBu,
        reversescale=False
        )
    )

fig.update_yaxes(title_text='Nome do Ponto de Parada', autorange='reversed')
fig.update_xaxes(title_text='Janela Horária', side='top')
fig.update_layout(showlegend=False, height=600, width=1000,
                 title='Mapa de calor de velocidades por direção e hora do dia')

fig.show()

# Exercício

-> Pra essa primeira semana, ignorar esta parte

Objetivo: Realizar uma análise exploratória e diagnóstica da estrutura e cobertura do sistema de transporte público do Rio de Janeiro a partir de dados GTFS (General Transit Feed Specification), utilizando ferramentas de análise espacial e processamento de redes.

Contexto:

Dados no formato GTFS representam informações detalhadas sobre a operação de sistemas de transporte público, como linhas, horários, trajetos, paradas e frequências. Ao interpretar esses dados espacialmente, podemos avaliar a cobertura, acessibilidade, redundância e conectividade das redes de transporte, além de detectar gargalos e áreas mal atendidas.

Neste exercício, você irá utilizar o conjunto GTFS da cidade do Rio de Janeiro para responder perguntas como:

    Quais áreas da cidade são melhor atendidas por transporte público?

    Existe sobreposição entre linhas ou concentração excessiva em certas zonas?

    Quais regiões têm maior tempo médio de espera ou menor frequência de serviço?

    Onde estão os principais gargalos da rede?

    Como está a distribuição das viagens ao longo do dia? Quais os horários de pico?

Você poderá explorar os dados usando a biblioteca gtfs_functions, que fornece ferramentas práticas para manipulação, visualização e análise de feeds GTFS. Além disso, outras bibliotecas úteis incluem:

    geopandas: manipulação espacial de paradas e trajetos;

    networkx: análise da rede de conexões entre paradas;

    matplotlib / contextily: visualizações georreferenciadas;

    pandas: operações tabulares nos dados do GTFS;

    PySAL: análises espaciais complementares (ex: autocorrelação geográfica).

→ Dada a estrutura do GTFS doRio, é possível que, antes de usar a GTFS Functions você precise primeiro processar o dado conforme explicado [aqui](https://gist.github.com/invisiblefunnel/6c9f3a9b537d3f0ad192c24777b6ae57).